## **Segmenting and Clustering Neighborhoods in Toronto**

In [0]:
pip install beautifulsoup4

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [0]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(url.content, "html.parser")

In [5]:
table_body=soup.find('tbody')
table_rows = table_body.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    l.append(row)

#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighborhood"])   
df.head()
#df.shape
 

,Postcode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [0]:
df2 = df[df["Borough"] != "Not assigned"]
df2 = df2.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df2['Neighborhood'] = np.where(df2['Neighborhood'] == 'Not assigned', df2["Borough"], df2['Neighborhood'])

In [7]:
df2.shape

(103, 3)

# **Add geographical coordinates**

In [14]:
geo = pd.read_csv('https://cocl.us/Geospatial_data')

<bound method NDFrame.head of     Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]>

In [17]:
geo.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df3 = pd.merge(df2, geo, on='Postcode', how='left')
df3

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
